In [1]:
import numpy as np
import pandas as pd

from desdeo_problem.testproblems.TestProblems import test_problem_builder

from desdeo_emo.EAs.NIMBUS_EA import NIMBUS_RVEA, NIMBUS_NSGAIII
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_emo.EAs.NSGAIII import NSGAIII

import plotly.express as ex
import plotly.graph_objects as go
from plotly.offline.offline import plot as plothtml

from desdeo_tools.scalarization.ASF import PointMethodASF as asf

from scipy.optimize import minimize, differential_evolution

In [2]:
n_obj = 5
n_var = 10
problem = test_problem_builder(name="DTLZ2", n_of_objectives=n_obj, n_of_variables=n_var)
problem.ideal = np.asarray([0] * n_obj)
problem.nadir = np.asarray([1]*n_obj)
results_folder_name = "./results/"

In [3]:
scalar = asf(ideal = problem.ideal, nadir=problem.nadir)

def get_min_scalar(scalar, response):
    def obj(y):
        if np.any(y<0):
            return 100
        if np.linalg.norm(y)<1:
            return 100
        else:
            return scalar(y, response)
    y = differential_evolution(obj, bounds=np.asarray((problem.ideal, problem.nadir)).T).x
    return scalar(y, response)

In [4]:
# a posteriori
a_post_rvea = RVEA(problem=problem, interact=False, n_gen_per_iter=150, n_iterations=4)
a_post_nsga = NSGAIII(problem=problem, interact=False, n_gen_per_iter=150, n_iterations=4)
# interactive
int_rvea = RVEA(problem=problem, interact=True, n_gen_per_iter=150)
int_nsga = NSGAIII(problem=problem, interact=True, n_gen_per_iter=150)
# New algorithm
nimb_rvea = NIMBUS_RVEA(problem, n_gen_per_iter=150)
nimb_nsga = NIMBUS_NSGAIII(problem, n_gen_per_iter=150)

num_f_evals = pd.DataFrame(columns=["RVEA", "NSGAIII", "iRVEA", "iNSGAIII", "NIMBUS-RVEA", "NIMBUS-NSGAIII"], index=[1,2,3,4])
num_solns = pd.DataFrame(columns=["RVEA", "NSGAIII", "iRVEA", "iNSGAIII", "NIMBUS-RVEA", "NIMBUS-NSGAIII"], index=[1,2,3,4])

In [5]:
i = 1
while a_post_rvea.continue_evolution():
    a_post_rvea.iterate()
    num_f_evals["RVEA"][i] = a_post_rvea._function_evaluation_count
    num_solns["RVEA"][i] = a_post_rvea.population.objectives.shape[0]
    i+=1

i = 1

while a_post_nsga.continue_evolution():
    a_post_nsga.iterate()
    num_f_evals["NSGAIII"][i] = a_post_nsga._function_evaluation_count
    num_solns["NSGAIII"][i] = a_post_nsga.population.objectives.shape[0]
    i+=1

In [6]:
"""responses = np.asarray([
    [0.9,0.2, 0.1],
    [0.5,0.5, 0.5],
    [0.1,0.3, 0.3],
    [0.2,0.5, 0.1],])"""
responses = np.random.rand(4,5)

In [7]:
_, pref_int_rvea = int_rvea.requests()
_, pref_int_nsga = int_nsga.requests()
_, pref_nimb_rvea = nimb_rvea.requests()
_, pref_nimb_nsga = nimb_nsga.requests()


for i, response in enumerate(responses):
    fig = go.Figure()
    optimality_fig = go.Figure()
    scalar_min_val = get_min_scalar(scalar, response)[0]
    
    pref_int_rvea.response = pd.DataFrame([response], columns=pref_int_rvea.content['dimensions_data'].columns)
    pref_int_nsga.response = pd.DataFrame([response], columns=pref_int_nsga.content['dimensions_data'].columns)
    pref_nimb_rvea.response = pd.DataFrame([response], columns=pref_nimb_rvea.content['dimensions_data'].columns)
    pref_nimb_nsga.response = pd.DataFrame([response], columns=pref_nimb_nsga.content['dimensions_data'].columns)
    
    _, pref_int_rvea = int_rvea.iterate(pref_int_rvea)
    num_f_evals["iRVEA"][i+1] = int_rvea._function_evaluation_count
    num_solns["iRVEA"][i+1] = int_rvea.population.objectives.shape[0]
    
    _, pref_int_nsga = int_nsga.iterate(pref_int_nsga)
    num_f_evals["iNSGAIII"][i+1] = int_nsga._function_evaluation_count
    num_solns["iNSGAIII"][i+1] = int_nsga.population.objectives.shape[0]
    
    _, pref_nimb_rvea = nimb_rvea.iterate(pref_nimb_rvea)
    num_f_evals["NIMBUS-RVEA"][i+1] = nimb_rvea._function_evaluation_count
    num_solns["NIMBUS-RVEA"][i+1] = nimb_rvea.population.objectives.shape[0]
    
    _, pref_nimb_nsga = nimb_nsga.iterate(pref_nimb_nsga)
    num_f_evals["NIMBUS-NSGAIII"][i+1] = nimb_nsga._function_evaluation_count
    num_solns["NIMBUS-NSGAIII"][i+1] = nimb_nsga.population.objectives.shape[0]
    
    fig.add_trace(go.Box(y=scalar(a_post_rvea.population.objectives, response),
                         boxpoints="all",
                         name="RVEA"))
    fig.add_trace(go.Box(y=scalar(a_post_nsga.population.objectives, response),
                         boxpoints="all",
                         name="NSGA-III"))
    fig.add_trace(go.Box(y=scalar(int_rvea.population.objectives, response),
                         boxpoints="all",
                         name="iRVEA"))
    fig.add_trace(go.Box(y=scalar(int_nsga.population.objectives, response),
                         boxpoints="all",
                         name="iNSGA-III"))
    fig.add_trace(go.Box(y=scalar(nimb_rvea.population.objectives, response),
                         boxpoints="all",
                         name="NIMBUS-RVEA"))
    fig.add_trace(go.Box(y=scalar(nimb_nsga.population.objectives, response),
                         boxpoints="all",
                         name="NIMBUS-NSGAIII"))
    fig.add_trace(go.Scatter(x= ["RVEA", "NIMBUS-NSGAIII"], y=[scalar_min_val]*2, mode="lines", name="Minimum asf value"))
    
    
    optimality_fig.add_trace(go.Box(y=np.linalg.norm(a_post_rvea.population.objectives, axis=1),
                         boxpoints="all",
                         name="RVEA"))
    optimality_fig.add_trace(go.Box(y=np.linalg.norm(a_post_nsga.population.objectives, axis=1),
                         boxpoints="all",
                         name="NSGA-III"))
    optimality_fig.add_trace(go.Box(y=np.linalg.norm(int_rvea.population.objectives, axis=1),
                         boxpoints="all",
                         name="iRVEA"))
    optimality_fig.add_trace(go.Box(y=np.linalg.norm(int_nsga.population.objectives, axis=1),
                         boxpoints="all",
                         name="iNSGA-III"))
    optimality_fig.add_trace(go.Box(y=np.linalg.norm(nimb_rvea.population.objectives, axis=1),
                         boxpoints="all",
                         name="NIMBUS-RVEA"))
    optimality_fig.add_trace(go.Box(y=np.linalg.norm(nimb_nsga.population.objectives, axis=1),
                         boxpoints="all",
                         name="NIMBUS-NSGAIII"))
    
    optimality_fig.add_trace(go.Scatter(x= ["RVEA", "NIMBUS-NSGAIII"], y=[1]*2, mode="lines", name="Norm=1"))
    
    fig.update_layout(xaxis_title="Methods",
                      yaxis_title="ASF Values",
                      title=f"ASF values of the solutions returned after {i+1} iterations")
    optimality_fig.update_layout(xaxis_title="Methods",
                                 yaxis_title="Norm Values",
                                 title=f"Norm values of the solutions returned after {i+1} iterations")
    
    plothtml(fig, filename=results_folder_name + f"ASF after iteration {i+1}.html", auto_open=False)
    plothtml(optimality_fig, filename=results_folder_name + f"Norm after iteration {i+1}.html", auto_open=False)

In [8]:
num_f_evals

,RVEA,NSGAIII,iRVEA,iNSGAIII,NIMBUS-RVEA,NIMBUS-NSGAIII
1,18852,19026,19318,19026,6830,7550
2,37752,37926,37562,37926,11346,15050
3,56652,56826,55564,56826,15618,22550
4,75552,75726,72324,75726,20298,30050


In [9]:
num_solns

,RVEA,NSGAIII,iRVEA,iNSGAIII,NIMBUS-RVEA,NIMBUS-NSGAIII
1,126,126,131,126,50,50
2,126,126,131,126,49,50
3,126,126,131,126,31,50
4,126,126,131,126,26,50
